In [53]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

from dotenv import load_dotenv

load_dotenv('./env/.env')

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
)

# 메세지 토큰 수의 최대값
# memory_key의 default값은 "history"이다
memory = ConversationSummaryBufferMemory(
    llm=llm, 
    max_token_limit=120, 
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

# 1. load_memory가 호출된다, invoke의 주석된 "chat_history" 와 같은 역할을 하는것이다 -> load_memory의 결과값이 chat_history input으로 들어간다
# 이 값은 Prompt의 variable_name에 들어간다
# 이렇게 RunnablePassthrough는 프롬프트가 format되기 전에 함수를 실행시키는걸 허락해주는 역할을 한다. 이 함수를 활용해 원하는 값을 변수에 할당할 수 있게 되는 것이다
# 그리고 이 변수들은 prompt로 전달된다
# assign을 할때 사용자의 input을 받게된다. 여기서의 input은 {"question": "My name is Michael"} 이다. 여기선 input을 활용하지 않으므로 _ 처리하자
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    # 사용자로 부터 질문을 받는 함수
    result =  chain.invoke({
        "question": question
    })
    # ex) result = AIMessage(content="Hello, I am an AI")
    memory.save_context({"input": question}, {"output": result.content})
    print(result)


invoke_chain("My name is Michael")




content='Hello Michael! How can I assist you today?'
